In [1]:
import sys
import time

import numpy as np
import  pandas as pd
import  torch
from torchvision import models, transforms,datasets

import os
import matplotlib.pyplot as plt
from collections import Counter


In [2]:
from src.dataset import MelanomaDataset


1

In [3]:
transform1=transforms.Compose([transforms.ToTensor(),transforms.Resize((380,380))])
transform2=transforms.ToTensor()

In [3]:
train_dataset1=datasets.ImageFolder(train,transform1)
train_dataset2=datasets.ImageFolder(train,transform2)

In [3]:
next(iter(train_dataset1))[1]

In [3]:
print(len(train_dataset1))

In [3]:
import cv2
# print(train_dataset1[0][0].shape)
# img=(train_dataset1[6])[0].numpy()
# img=np.transpose(img, (1, 2, 0))
# print(img.shape)
# plt.imshow(img)

for i in range(10):

        img1=(train_dataset1[i])[0].numpy()
        img1=np.transpose(img1, (1, 2, 0))
        print(img1.shape)
        plt.imshow(img1)
        plt.show()
        print("..............after................")
        img2=(train_dataset2[i])[0].numpy()
        img2=np.transpose(img1, (1, 2, 0))
        print(img2.shape)
        plt.imshow(img1)
        plt.show()









In [ ]:
length=[]
width=[]
cnt=0.0
total=float(len(train_dataset1))
for i in range(len(train_dataset1)):
        length.append(train_dataset1[i][0].shape[1])
        width.append(train_dataset1[i][0].shape[2])
        sys.stdout.flush()
        sys.stdout.write("\r data loaded = "+str(cnt*100/total)[:8]+" %")
        cnt+=1.0




In [0]:
width

In [3]:
length.sort()
width.sort()

In [113]:
common_length=Counter(length)
common_length.most_common()

In [113]:

common_width=Counter(width)
common_width.most_common()



In [4]:
from torch.utils.data import DataLoader

In [11]:
dataset_path="..\\dataset\\raw"
transform1=transforms.Compose([transforms.ToTensor(),transforms.Resize((380,380))])

train=os.path.join(dataset_path,'train\\')
test=os.path.join(dataset_path,'test\\')
class_name_idx={"melanoma":1,"nevus":0,"seborrheic_keratosis":0}
train_data=MelanomaDataset(train,class_name_idx,transform1)

In [12]:
train_loader=DataLoader(train_data,shuffle=True,batch_size=4)

In [13]:
from src.model import SkinCancerModel

In [14]:
model=SkinCancerModel(True,"efficientnet_b4")


In [17]:
model

SkinCancerModel(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivati

In [16]:
for i in model.classifier.parameters():
        i.requires_grad=False

In [10]:
model.save_weights("weights.pt")

In [8]:
model.load_local_weights("weights.pt")

In [19]:
from torch.optim import RMSprop
from torch.nn import MSELoss
import time
optimizer=RMSprop(model.parameters(),lr=0.01)
loss_function=MSELoss()

In [20]:
epochs=1000
batch_size=train_loader.batch_size
no_batches=len(train_loader)
dataset_size=float(len(train_loader.dataset))
model.cuda()

for e in range(epochs):
        loss_sum=0.0
        cnt=0.0
        time_sum=0.0
        for img,value in train_loader:
                ts=time.time()
                optimizer.zero_grad()
                img=img.cuda()
                output=model(img)
                value=value.cuda()

                loss=loss_function(output,value)

                loss.backward()
                optimizer.step()
                loss_sum+=loss.item()*batch_size
                cnt+=1.0

                finished=int((cnt*10)/no_batches)
                remaining=10-finished
                te=time.time()
                time_sum+=(te-ts)
                avg_time=time_sum/cnt
                time_remaing=avg_time*(no_batches-cnt)
                sys.stdout.write("\r ephoch "+str(e+1)+" ["+str("="*int((cnt*10)/no_batches)+str("."*remaining)+"] time remaining = "+str(time_remaing/60.0)[:8]))
        print(f"epoch {e+1} loss ={loss_sum/dataset_size}")

 ephoch 1 [==========] time remaining = 0.0epoch 1 loss =0.1572503129001707
 ephoch 2 [==========] time remaining = 0.0epoch 2 loss =0.15541785646229983
 ephoch 3 [==========] time remaining = 0.0epoch 3 loss =0.15631725982204078
 ephoch 4 [..........] time remaining = 9.152931


KeyboardInterrupt



In [22]:
   torch.cuda.empty_cache()